In [ ]:
!nvidia-smi

Wed Dec 21 00:38:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    28W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install sentence-transformers transformers ipyplot  tinydb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.9 MB/s 
     |████████████████████████████████| 5.8 MB 10.3 MB/s 
     |████████████████████████████████| 1.3 MB 46.2 MB/s 
     |████████████████████████████████| 182 kB 80.7 MB/s 
     |████████████████████████████████| 7.6 MB 60.7 MB/s 
     |████████████████████████████████| 1.6 MB 8.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=6e53de0808e1be92b46d9fde74cffce77c704334f7f51840424bd593a5275a84
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [ ]:
!python -m spacy download pt_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.0 MB 2.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [ ]:
import pt_core_news_sm
nlp = pt_core_news_sm.load()

def lemmatize(sentence):
  doc = nlp(sentence.lower())
  pos_ = []
  lemmas = []
  tokens = []
  for t in doc:
    tokens.append({
        "text": t.text,
        "lemma": t.lemma_,
        'pos': t.pos_
    })
  return tokens
lemmatize('eu gosto de ver tevê.')

[{'text': 'eu', 'lemma': 'eu', 'pos': 'PRON'},
 {'text': 'gosto', 'lemma': 'gostar', 'pos': 'VERB'},
 {'text': 'de', 'lemma': 'de', 'pos': 'SCONJ'},
 {'text': 'ver', 'lemma': 'ver', 'pos': 'VERB'},
 {'text': 'tevê', 'lemma': 'tever', 'pos': 'X'},
 {'text': '.', 'lemma': '.', 'pos': 'PUNCT'}]

In [ ]:
docs = list(nlp.pipe(["eu quero sair","eu amo você"]))
for doc in docs:
  print(doc)

eu quero sair
eu amo você


In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertForMaskedLM
import torch

BASE_MODEL = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = BertForMaskedLM.from_pretrained(BASE_MODEL,output_hidden_states=True)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)


Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/210k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import requests,json

res = requests.get("https://api.arasaac.org/api/pictograms/all/br")
data = res.json()

n_data = {}
for i, item in enumerate(data):
    n_data[item['_id']] = item

json.dump({
    "pictogram":n_data
}, open("pictograms.json","w"))


In [ ]:
from nltk.tokenize import MWETokenizer
vocab = []
for item in data:
  for k in item['keywords']:
    vocab.append(tuple(k["keyword"].lower().split(" ")))
MWEtokenizer = MWETokenizer(vocab)

In [ ]:
MWEtokenizer.tokenize('Vamos tomar café da manhã'.lower().split())

['vamos', 'tomar', 'café_da_manhã']

In [ ]:
from tinydb import TinyDB,Query
import re
Pictogram = Query()
Keyword = Query()
db = TinyDB("pictograms.json")
picograms = db.table("pictogram")


pos_mapping = {
    "VERB": 3,
    "AUX": 3,
    'ADJ':4,
    "DET":6,
    "NOUN":2,
    "PRON":1,
    "PROPN":1,
    "ADP":6,
    "ADV":6,
    "CCONJ":6,
    "SCONJ":6,
    "NUM":6,
    "PUNCT":6,
    "SPACE":2,
    "X":2
}

def get_alternatives(word,pos=None, best=True,allow_simple_search=True):
  query = None
  if pos is not None:
    pos_ = pos_mapping[pos]
    if best:
      query = Pictogram.keywords.any(Keyword.keyword==word) & Pictogram.keywords.any(Keyword.type == pos_)
    else:
      query = Pictogram.keywords.any(Keyword.keyword.matches(word, flags=re.IGNORECASE)) & Pictogram.keywords.any(Keyword.type == pos_)
  else:
    if best:
      query = Pictogram.keywords.any(Keyword.keyword==word)
    else:
      query = Pictogram.keywords.any(Keyword.keyword.matches(word, flags=re.IGNORECASE))
  # return query
    
  alternatives = picograms.search(query)
  if len(alternatives) ==0 and best and allow_simple_search:
    return get_alternatives(word,best=False)
  return alternatives

get_alternatives("eu quero comer")


[]

In [ ]:
import torch
def get_target_embeddings(sentence, target):
  tokenized = tokenizer(sentence,return_tensors="pt")
  target_tokens = tokenizer.encode(target)
  target_position = []
  t = tokenized["input_ids"][0]
  q = torch.tensor(target_tokens[1:-1])
  target_position = torch.isin(t, q).nonzero()
  target_position.reshape(target_position.size()[0])
  tokenized = tokenized.to(device)
  outputs = model(**tokenized)
  hidden_states = torch.stack(list(outputs['hidden_states'])[-4:]).sum(0)
  # return hidden_states
  return hidden_states[0,target_position,:].detach().cpu().mean(0)
  return outputs["hidden_states"][-1][0,target_position,:].detach().cpu().mean(0)
# res = get_target_embeddings("I would love some coffee", "coffe")
# res.size()

def get_definition_emb(sentence):
  tokenized = tokenizer(sentence,return_tensors="pt")
  tokenized = tokenized.to(device)
  outputs = model(**tokenized)
  hidden_states = torch.stack(list(outputs['hidden_states'])[-4:]).sum(0)
  return hidden_states[0,0,:].detach().cpu()
  # return outputs["hidden_states"][-1][0,0,:].detach()
# res = get_target_embeddings("I would love some coffee", "coffe")
# res.size()

In [ ]:
def get_by_id(id_):
  res = requests.get("https://api.arasaac.org/api/pictograms/br/{0}".format(id_))
  return res.json()
# get_by_id(6632)

In [ ]:
from tqdm import tqdm
pictogram_ids=[]
pictogram_defs=[]

for item in tqdm(data):
  pictogram_ids.append(item["_id"])
  defs = []
  for k in item['keywords']:
    if "meaning" in k:
      # print(k['meaning'])
      try:
        defs.append(k['meaning'].encode('cp1252').decode('utf-8'))
      except:
        defs.append(k['meaning'])
    try:
      defs.append(k['keyword'].encode('cp1252').decode('utf-8'))
    except:
      defs.append(k['keyword'])

  s = "; ".join(defs)
  pictogram_defs.append(s)

100%|██████████| 12785/12785 [00:00<00:00, 240088.19it/s]


In [ ]:
tokenized = tokenizer(pictogram_defs,max_length=225,padding=True,truncation=True,return_tensors="pt")

In [ ]:
embs = torch.zeros(len(pictogram_defs),768)

batch_size = 32
for i in tqdm(range(0,len(pictogram_defs),batch_size)):
  docs = pictogram_defs[i:i+batch_size]
  tokenized = tokenizer(docs,max_length=225,padding=True,truncation=True,return_tensors="pt")
  tokenized = tokenized.to(device)
  outputs = model(**tokenized)
  hidden_states = torch.stack(list(outputs['hidden_states'])[-4:]).sum(0)
  alternatives_embs = hidden_states[:,0,:].detach().cpu()
  embs[i:i+batch_size] = alternatives_embs

100%|██████████| 400/400 [01:03<00:00,  6.35it/s]


In [ ]:
import numpy as np

from sklearn.neighbors import NearestNeighbors

from sklearn.metrics.pairwise import cosine_similarity
def get_pictogram(sentence, target,allow_simple_search=True, pos=None):
  if target.lower() == "eu":
    return get_by_id(6632)
  elif target.lower() == "não":
    return get_by_id(5526)
  # print(target,pos,allow_simple_search)

  sentence = sentence
  sentence_code = get_target_embeddings(sentence,target)

  alternatives = get_alternatives(target,pos=pos,allow_simple_search=allow_simple_search)
  if len(alternatives) == 1:
    return alternatives[0]
  elif len(alternatives) == 0:
    return None
  
  ids_ = [a['_id'] for a in alternatives]
  indices = [pictogram_ids.index(id_) for id_ in ids_]

  alternatives_embs = embs[indices]
  neigh = NearestNeighbors(n_neighbors=1,algorithm='ball_tree')

  neigh.fit(alternatives_embs)
  _, indices = neigh.kneighbors(sentence_code)
  return alternatives[indices[0][0]]

get_pictogram("Eu quero comer batatas fritas", "comer")

{'schematic': False,
 'sex': False,
 'violence': False,
 'aac': False,
 'aacColor': False,
 'skin': True,
 'hair': True,
 'downloads': 0,
 'categories': ['verb', 'event', 'feeding'],
 'synsets': ['01170315-v'],
 'tags': ['communication', 'language', 'verb', 'event', 'feeding'],
 '_id': 4593,
 'created': '2008-04-28T13:08:14.000Z',
 'lastUpdated': '2021-02-28T04:06:46.025Z',
 'keywords': [{'meaning': 'Dine.',
   'type': 3,
   'keyword': 'jantar',
   'hasLocution': False},
  {'keyword': 'comer', 'hasLocution': False, 'type': 3}]}

In [ ]:
def transform_sentence(sentence):
  tokens = lemmatize(sentence)
  n_sentence = " ".join([t['lemma'] for t in tokens])
  pictograms = []
  skip = 0
  for i in range(len(tokens)):
    t = tokens[i]['lemma']
    pictogram = get_pictogram(n_sentence, t,pos=tokens[i]['pos'])
    if pictogram is not None:
      pictogram['tokens'] = [tokens[i]]
      pictogram['caption'] = t  
      pictograms.append({
        "id": pictogram["_id"],
        "caption": t,
        "tokens": [tokens[i]]
      })
  return pictograms
transform_sentence("Eu quero comer batatas fritas")

[{'id': 6632,
  'caption': 'eu',
  'tokens': [{'text': 'eu', 'lemma': 'eu', 'pos': 'PRON'}]},
 {'id': 36518,
  'caption': 'querer',
  'tokens': [{'text': 'quero', 'lemma': 'querer', 'pos': 'VERB'}]},
 {'id': 4593,
  'caption': 'comer',
  'tokens': [{'text': 'comer', 'lemma': 'comer', 'pos': 'VERB'}]},
 {'id': 2503,
  'caption': 'batata',
  'tokens': [{'text': 'batatas', 'lemma': 'batata', 'pos': 'NOUN'}]},
 {'id': 3325,
  'caption': 'frita',
  'tokens': [{'text': 'fritas', 'lemma': 'frita', 'pos': 'ADJ'}]}]

In [ ]:
from io import BytesIO
from PIL import Image
import ipyplot

def show_sentence(pictograms):
  images = []
  labels = []
  texts = []
  for card in pictograms:
    if card is not None:
      url = "https://api.arasaac.org/api/pictograms/{0}?download=false".format(card['_id'])
      response = requests.get(url)
      images.append(Image.open(BytesIO(response.content)))
      labels.append(card['caption'])
  ipyplot.plot_images(images,labels)


        WARNING! Google Colab Environment detected!
        You might encounter issues while running in Google Colab environment.
        If images are not displaying properly please try setting `force_b64` param to `True`.
        


In [ ]:
!gdown https://drive.google.com/uc?id=1LPnJFvNUlVvhGaFRAa95G5bgXfJvpj3m

In [ ]:
import pandas as pd

df = pd.read_csv("./not_offensive_lower_perplexities_no_large_13jul.csv")

In [ ]:
f = open("pictogram_sentences.json",'w')

In [ ]:
import json
from tqdm import tqdm
pictogram_sentences = []
sentences = df['sentences'].to_list()
for l in tqdm(sentences):
  pics = transform_sentence(l.strip())
  pictogram_sentences.append(pics)
  json.dump(pictogram_sentences,f)

  0%|          | 19/13835 [00:38<7:40:56,  2.00s/it]


KeyboardInterrupt: ignored